In [1]:
#Import Libraries
import pandas as pd
import plotly.express as px
from pathlib import Path
import numpy as np
import re

In [2]:
# Load CSVs
books_path = Path('Resources/books.csv')
ratings_path = Path('Resources/ratings.csv')
users_path = Path('Resources/users.csv')

In [3]:
# Check CSV encoding
with open(books_path) as f:
    print(f)

<_io.TextIOWrapper name='Resources/books.csv' mode='r' encoding='UTF-8'>


In [4]:
# Read CSV with latin-1 encoding due to variety of latin characters
# And assign data types for consistency
# Columns ISBN and Year-Of-Publication cannot be read in as int data types. For example, ISBN contains alphanumeric values. Year-Of-Publication column contains alphabetical values and not exclusively number values. This will be explored below.

books_df = pd.read_csv(
    books_path,
    sep=';',
    on_bad_lines='warn',
    encoding='latin-1', 
    dtype={
        'ISBN': str,
        'Book-Title': str,
        'Book-Author': str,
        'Year-Of-Publication': str,
        'Publisher': str,
        'Image-URL-S': str,
        'Image-URL-M': str,
        'Image-URL-L': str,
        }
    )

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


### Note: The warning above indicates rows with inconsistent columns to the rest of the books_df dataset and will be omitted from the cleaned DataFrame

In [5]:
# Read CSV
ratings_df = pd.read_csv(ratings_path, sep=';', on_bad_lines='warn', encoding='latin-1')

In [6]:
# Read CSV
users_df = pd.read_csv(users_path, sep=';', on_bad_lines='warn', encoding='latin-1')

In [7]:
# Check head
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
#Check columns
books_df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [9]:
# Check head
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [10]:
# Check head
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
# Check shape
print('books_df', books_df.shape)
print('ratings_df', ratings_df.shape)
print('users_df', users_df.shape)

books_df (271360, 8)
ratings_df (1149780, 3)
users_df (278858, 3)


In [12]:
# Check nulls
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [13]:
# Check nulls
ratings_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [14]:
# Check nulls
users_df.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [15]:
# Check duplicates
books_df.duplicated().sum()

0

In [16]:
# Check duplicates
ratings_df.duplicated().sum()

0

In [17]:
# Check duplicates
users_df.duplicated().sum()

0

In [18]:
# Define function to locate index of values (Source: https://www.geeksforgeeks.org/find-location-of-an-element-in-pandas-dataframe-in-python/)
def get_indexes(object, value):
     
    # Empty list
    position_list = []
     
    # isin() method will return a DataFrame with
    # boolean values, True at the positions   
    # where element exists
    result = object.isin([value])
     
    # any() method will return
    # a boolean series
    series_object = result.any()
 
    # Get list of column names where
    # element exists
    column_names = list(series_object[series_object == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in column_names:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            position_list.append((row, col))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return position_list

In [19]:
# Apply get_indexes function to locate NaN values
positions = get_indexes(books_df, np.nan)
print('Index positions of NaN in books_df: ')
for i in range (len(positions)):
    print(positions[i])

Index positions of NaN in books_df: 
(187689, 'Book-Author')
(128890, 'Publisher')
(129037, 'Publisher')
(209538, 'Image-URL-L')
(220731, 'Image-URL-L')
(221678, 'Image-URL-L')


# Manage Null Values

## Books DataFrame

In [20]:
# Check NaN to correct missing values
books_df.iloc[187689]

ISBN                                                          9627982032
Book-Title             The Credit Suisse Guide to Managing Your Perso...
Book-Author                                                          NaN
Year-Of-Publication                                                 1995
Publisher                                 Edinburgh Financial Publishing
Image-URL-S            http://images.amazon.com/images/P/9627982032.0...
Image-URL-M            http://images.amazon.com/images/P/9627982032.0...
Image-URL-L            http://images.amazon.com/images/P/9627982032.0...
Name: 187689, dtype: object

In [21]:
# Check NaN to correct missing values
books_df.iloc[128890]

ISBN                                                          193169656X
Book-Title                                                   Tyrant Moon
Book-Author                                              Elaine Corvidae
Year-Of-Publication                                                 2002
Publisher                                                            NaN
Image-URL-S            http://images.amazon.com/images/P/193169656X.0...
Image-URL-M            http://images.amazon.com/images/P/193169656X.0...
Image-URL-L            http://images.amazon.com/images/P/193169656X.0...
Name: 128890, dtype: object

In [22]:
# Check NaN to correct missing values
books_df.iloc[129037]

ISBN                                                          1931696993
Book-Title                                               Finders Keepers
Book-Author                                              Linnea Sinclair
Year-Of-Publication                                                 2001
Publisher                                                            NaN
Image-URL-S            http://images.amazon.com/images/P/1931696993.0...
Image-URL-M            http://images.amazon.com/images/P/1931696993.0...
Image-URL-L            http://images.amazon.com/images/P/1931696993.0...
Name: 129037, dtype: object

In [23]:
# Check NaN to correct missing values
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: Creating the X-Men, How It All Beg...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/078946697X.0...
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L                                                          NaN
Name: 209538, dtype: object

In [24]:
# Check NaN to correct missing values
books_df.iloc[220731]

ISBN                                                          2070426769
Book-Title             Peuple du ciel, suivi de 'Les Bergers\";Jean-M...
Book-Author                                                         2003
Year-Of-Publication                                            Gallimard
Publisher              http://images.amazon.com/images/P/2070426769.0...
Image-URL-S            http://images.amazon.com/images/P/2070426769.0...
Image-URL-M            http://images.amazon.com/images/P/2070426769.0...
Image-URL-L                                                          NaN
Name: 220731, dtype: object

In [25]:
# Check NaN to correct missing values
books_df.iloc[221678]

ISBN                                                          0789466953
Book-Title             DK Readers: Creating the X-Men, How Comic Book...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/0789466953.0...
Image-URL-S            http://images.amazon.com/images/P/0789466953.0...
Image-URL-M            http://images.amazon.com/images/P/0789466953.0...
Image-URL-L                                                          NaN
Name: 221678, dtype: object

In [26]:
# Add missing author and publisher values as per research on Amazon and bookfinder.com via ISBN search
books_df.at[187689, 'Book-Author'] = 'Larissa Anne Downes, Editor'
books_df.at[128890, 'Publisher'] = 'CreateSpace Independent Publishing Platform'
books_df.at[129037, 'Publisher'] = 'NovelBooks Inc'

In [27]:
# Check corrected value
books_df.iloc[187689]

ISBN                                                          9627982032
Book-Title             The Credit Suisse Guide to Managing Your Perso...
Book-Author                                  Larissa Anne Downes, Editor
Year-Of-Publication                                                 1995
Publisher                                 Edinburgh Financial Publishing
Image-URL-S            http://images.amazon.com/images/P/9627982032.0...
Image-URL-M            http://images.amazon.com/images/P/9627982032.0...
Image-URL-L            http://images.amazon.com/images/P/9627982032.0...
Name: 187689, dtype: object

In [28]:
# Check corrected value
books_df.iloc[6452]

ISBN                                                          0307121259
Book-Title                             I Am a Bunny (Golden Sturdy Book)
Book-Author                                                     O. Risom
Year-Of-Publication                                                 1963
Publisher                                                   Golden Books
Image-URL-S            http://images.amazon.com/images/P/0307121259.0...
Image-URL-M            http://images.amazon.com/images/P/0307121259.0...
Image-URL-L            http://images.amazon.com/images/P/0307121259.0...
Name: 6452, dtype: object

In [29]:
# Check NaN to correct missing value
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: Creating the X-Men, How It All Beg...
Book-Author                                                         2000
Year-Of-Publication                                    DK Publishing Inc
Publisher              http://images.amazon.com/images/P/078946697X.0...
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L                                                          NaN
Name: 209538, dtype: object

In [30]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[209538] = books_df.iloc[209538].shift()
books_df.at[209538, 'ISBN'] = '078946697X'
books_df.at[209538, 'Book-Title'] = 'DK Readers: The Story of the X-Men, How It All Began (Level 4: Proficient Readers)'
books_df.at[209538, 'Book-Author'] = 'Michael Teitelbaum'
books_df.at[209538, 'Publisher'] = 'DK Publishing Inc'
books_df.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: The Story of the X-Men, How It All...
Book-Author                                           Michael Teitelbaum
Year-Of-Publication                                                 2000
Publisher                                              DK Publishing Inc
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L            http://images.amazon.com/images/P/078946697X.0...
Name: 209538, dtype: object

In [31]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[220731] = books_df.iloc[220731].shift()
books_df.at[220731, 'ISBN'] = '2070426769'
books_df.at[220731, 'Book-Title'] = 'Peuple du ciel suivi de les Bergers'
books_df.at[220731, 'Book-Author'] = 'Jean-Marie Gustave'
books_df.at[220731, 'Publisher'] = 'Gallimard'
books_df.iloc[220731]

ISBN                                                          2070426769
Book-Title                           Peuple du ciel suivi de les Bergers
Book-Author                                           Jean-Marie Gustave
Year-Of-Publication                                                 2003
Publisher                                                      Gallimard
Image-URL-S            http://images.amazon.com/images/P/2070426769.0...
Image-URL-M            http://images.amazon.com/images/P/2070426769.0...
Image-URL-L            http://images.amazon.com/images/P/2070426769.0...
Name: 220731, dtype: object

In [32]:
# Shift and adjust values as per research on Amazon and bookfinder.com via ISBN search
books_df.iloc[221678] = books_df.iloc[221678].shift()
books_df.at[221678, 'ISBN'] = '2070426769'
books_df.at[221678, 'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books_df.at[221678, 'Book-Author'] = 'James Buckley'
books_df.at[221678, 'Publisher'] = 'DK Publishing Inc'
books_df.iloc[221678]

ISBN                                                          2070426769
Book-Title             DK Readers: Creating the X-Men, How Comic Book...
Book-Author                                                James Buckley
Year-Of-Publication                                                 2000
Publisher                                              DK Publishing Inc
Image-URL-S            http://images.amazon.com/images/P/0789466953.0...
Image-URL-M            http://images.amazon.com/images/P/0789466953.0...
Image-URL-L            http://images.amazon.com/images/P/0789466953.0...
Name: 221678, dtype: object

In [33]:
# Check books_df row count
books_row_count = len(books_df)
books_row_count

271360

### Check ISBN alphanemuric count to determine if column should remain as string data type

In [34]:
# Define function to detect alphanumeric values
# Source - https://www.geeksforgeeks.org/how-to-check-string-is-alphanumeric-or-not-using-regular-expression/

# Function to check string
# is alphanumeric or not
def isAlphaNumeric(str):

	# Regex to check string is
	# alphanumeric or not.
	regex = "^(?=.*[a-zA-Z])(?=.*[0-9])[A-Za-z0-9]+$"

	# Compile the ReGex
	p = re.compile(regex)

	# If the string is empty
	# return false
	if(str == None):
		return False

	# Return if the string
	# matched the ReGex
	if(re.search(p, str)):
		return True
	else:
		return False


In [35]:
# Store ISBN values to list
ISBN_list = books_df['ISBN'].values.tolist()

In [36]:
# List comprehension for Alphanumeric check on ISBN_List
alphanumeric_list = [isAlphaNumeric(i) for i in ISBN_list]

In [37]:
# Count for True
alphanumeric_list.count(True)

22330

In [38]:
# Count for False
alphanumeric_list.count(False)

249030

### Given the results, ISBN will remain as string data type

## Check Unique values for Year-of-Publication column

In [39]:
# Check unique Year-of-Publication values
print(sorted(books_df['Year-Of-Publication'].unique()))

['0', '1376', '1378', '1806', '1897', '1900', '1901', '1902', '1904', '1906', '1908', '1909', '1910', '1911', '1914', '1917', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2008', '2010', '2011', '2012', '2020', '2021', '2024', '2026', '2030', '2037', '2038', '2050']


## There are some Year-of-Publication values set into the future and some at zero. We will remove these from the dataset.

## Future opportunity to programmatically impute values

In [40]:
# Get indexes of 0 value in Year-of-Publication
year_zero_positions = get_indexes(books_df, '0')
print('Index positions of 0 in books_df: ')
for i in range (len(year_zero_positions)):
    print(year_zero_positions[i])

Index positions of 0 in books_df: 
(176, 'Year-Of-Publication')
(188, 'Year-Of-Publication')
(288, 'Year-Of-Publication')
(351, 'Year-Of-Publication')
(542, 'Year-Of-Publication')
(641, 'Year-Of-Publication')
(646, 'Year-Of-Publication')
(728, 'Year-Of-Publication')
(732, 'Year-Of-Publication')
(735, 'Year-Of-Publication')
(797, 'Year-Of-Publication')
(810, 'Year-Of-Publication')
(820, 'Year-Of-Publication')
(924, 'Year-Of-Publication')
(1058, 'Year-Of-Publication')
(1095, 'Year-Of-Publication')
(1165, 'Year-Of-Publication')
(1227, 'Year-Of-Publication')
(1253, 'Year-Of-Publication')
(1261, 'Year-Of-Publication')
(1263, 'Year-Of-Publication')
(1381, 'Year-Of-Publication')
(1390, 'Year-Of-Publication')
(1448, 'Year-Of-Publication')
(1623, 'Year-Of-Publication')
(1631, 'Year-Of-Publication')
(1637, 'Year-Of-Publication')
(1734, 'Year-Of-Publication')
(1743, 'Year-Of-Publication')
(1814, 'Year-Of-Publication')
(1870, 'Year-Of-Publication')
(1905, 'Year-Of-Publication')
(1913, 'Year-Of-Pub

In [41]:
# Get count of Year-of-Publication with value of 0
year_zero_error = len(year_zero_positions)
year_zero_error

4618

In [42]:
# Get indexes of x value in Year-of-Publication
year_2050_positions = get_indexes(books_df, '2050')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2050_positions)):
    print(year_2050_positions[i])

Index positions of 0 in books_df: 
(80264, 'Year-Of-Publication')
(97826, 'Year-Of-Publication')


In [43]:
# Get count of Year-of-Publication errors
year_2050_error = len(year_2050_positions)
year_2050_error

2

In [44]:
# Get indexes of x value in Year-of-Publication
year_2038_positions = get_indexes(books_df, '2038')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2038_positions)):
    print(year_2038_positions[i])

Index positions of 0 in books_df: 
(116053, 'Year-Of-Publication')


In [45]:
# Get count of Year-of-Publication errors
year_2038_error = len(year_2038_positions)
year_2038_error

1

In [46]:
# Get indexes of x value in Year-of-Publication
year_2037_positions = get_indexes(books_df, '2037')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2037_positions)):
    print(year_2037_positions[i])

Index positions of 0 in books_df: 
(255409, 'Year-Of-Publication')


In [47]:
# Get count of Year-of-Publication errors
year_2037_error = len(year_2037_positions)
year_2037_error

1

In [48]:
# Get indexes of x value in Year-of-Publication
year_2030_positions = get_indexes(books_df, '2030')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2030_positions)):
    print(year_2030_positions[i])

Index positions of 0 in books_df: 
(37487, 'Year-Of-Publication')
(55676, 'Year-Of-Publication')
(78168, 'Year-Of-Publication')
(192993, 'Year-Of-Publication')
(228173, 'Year-Of-Publication')
(240169, 'Year-Of-Publication')
(260974, 'Year-Of-Publication')


In [49]:
# Get count of Year-of-Publication errors
year_2030_error = len(year_2030_positions)
year_2030_error

7

In [50]:
# Get indexes of x value in Year-of-Publication
year_2026_positions = get_indexes(books_df, '2026')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2026_positions)):
    print(year_2026_positions[i])

Index positions of 0 in books_df: 
(118294, 'Year-Of-Publication')


In [51]:
# Get count of Year-of-Publication errors
year_2026_error = len(year_2026_positions)
year_2026_error

1

In [52]:
# Get indexes of x value in Year-of-Publication
year_2024_positions = get_indexes(books_df, '2024')
print('Index positions of 0 in books_df: ')
for i in range (len(year_2024_positions)):
    print(year_2024_positions[i])

Index positions of 0 in books_df: 
(142716, 'Book-Title')
(246842, 'Year-Of-Publication')


In [53]:
# Get count of Year-of-Publication errors
year_2024_error = len(year_2024_positions)
year_2024_error

2

In [54]:
# Total count of Year-of-Publication data errors
sum_year_errors = year_zero_error + year_2050_error + year_2038_error + year_2037_error + year_2030_error + year_2026_error + year_2024_error
print(f'The total count of Year-of-Publication errors is {sum_year_errors}.')

The total count of Year-of-Publication errors is 4632.


In [55]:
# Drop all entries with incorrect Year-of-Publication dates
books_df = books_df[books_df['Year-Of-Publication'] != '0']
books_df = books_df[books_df['Year-Of-Publication'] != '2050']
books_df = books_df[books_df['Year-Of-Publication'] != '2038']
books_df = books_df[books_df['Year-Of-Publication'] != '2037']
books_df = books_df[books_df['Year-Of-Publication'] != '2030']
books_df = books_df[books_df['Year-Of-Publication'] != '2026']
books_df = books_df[books_df['Year-Of-Publication'] != '2024']

In [56]:
# Check books_df row count after drop
books_row_count2 = len(books_df)
books_row_count2

266729

In [57]:
# Check unique Year-of-Publication values
print(sorted(books_df['Year-Of-Publication'].unique()))

['1376', '1378', '1806', '1897', '1900', '1901', '1902', '1904', '1906', '1908', '1909', '1910', '1911', '1914', '1917', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2008', '2010', '2011', '2012', '2020', '2021']


In [58]:
# Subtract books_df row count prior to dropping Year-of-Publication errors to check for consistency
books_df_difference = books_row_count - books_row_count2
books_df_difference

4631

In [59]:
# Difference between sum_year_errors and books_df_difference
sum_year_errors - books_df_difference

1

### We know the value '2024' is found at (142716, 'Book-Title') from calculating the variable year_2024_positions. 

In [60]:
# Convert string to int data type
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(np.int32)

In [61]:
# Drop Image-URL columns since they do not add value for recommendation system
# books_df.drop(books_df.iloc[:, 5:8], inplace=True, axis=1)

In [62]:
# Write cleaned DataFrame to csv
# books_df.to_csv('Resources/books_cleaned.csv')

## Users DataFrame - Manage Nulls and check unique values

In [63]:
# Check unique age values
print(sorted(users_df['Age'].unique()))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 223.0, 226.0

In [64]:
# Check users_df row count
len(users_df)

278858

### It doesn't make much sense to have age values that are < 5 and > 90

In [65]:
# Check NaN values
users_df['Age'].isna().sum()

110762

In [66]:
# Replace Age values below 5 and above 90 with NaN values source - The idea to manage these values this way originates from https://www.kaggle.com/code/saurabhbagchi/recommender-system-for-books
users_df.loc[(users_df['Age'] > 90) | (users_df['Age'] < 5), 'Age'] = np.nan

In [67]:
# Check Age mean
users_df['Age'].mean()

34.72384041634689

In [68]:
# Impute Age column NaN values with mean
users_df['Age'].fillna(users_df['Age'].mean(), inplace=True)

In [69]:
# Convert Age column data type to int
users_df['Age'] = users_df['Age'].astype(np.int32)

In [70]:
# Check unique age values
print(sorted(users_df['Age'].unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


In [71]:
# Write cleaned DataFrame to csv
#users_df.to_csv('Resources/users_cleaned.csv')

## Check Ratings DataFrame for unique values

In [72]:
# Check Book-Rating values
print(sorted(ratings_df['Book-Rating'].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [73]:
# Check ratings_df row count
len(ratings_df)

1149780